<a href="https://colab.research.google.com/github/lidorsandak/ad_ML_Competition_m5/blob/main/optimal_reconciliation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install datasetsforecast hierarchicalforecast neuralforecast statsforecast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.2/263.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 32.9 MB/s eta 0:00:00


In [6]:
import pandas as pd

from datasetsforecast.hierarchical import HierarchicalData
from hierarchicalforecast.utils import aggregate, HierarchicalPlot
from neuralforecast.utils import augment_calendar_df
from utilsforecast.plotting import plot_series

In [14]:
import pandas as pd

#obtain hierarchical dataset
from datasetsforecast.hierarchical import HierarchicalData

# compute base forecast no coherent
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive

#obtain hierarchical reconciliation methods and evaluation
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import evaluate
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut
from utilsforecast.losses import mse

In [13]:
# # --- Correct Imports for hierarchicalforecast v1.0.0 ---

# from hierarchicalforecast.core import HierarchicalForecast
# from hierarchicalforecast.utils import is_strictly_hierarchical
# from hierarchicalforecast.reconciliation import MinT

# import lightgbm as lgb
# import pandas as pd
# import numpy as np

# print("Successfully imported HierarchicalForecast and dependencies!")

ImportError: cannot import name 'HierarchicalForecast' from 'hierarchicalforecast.core' (/usr/local/lib/python3.12/dist-packages/hierarchicalforecast/core.py)

# M5 Forecasting Challenge: Predicting Daily Walmart Revenue

**Course:** Advanced Machine Learning - Final Assignment

---

### Project Objective

The goal of this project is to develop a high-performance forecasting model to predict the daily revenue for 10 individual Walmart stores and an aggregate of all stores. The project is inspired by the M5 Forecasting Competition and uses a modified version of its dataset.

Success is measured by the Root Mean Squared Error (RMSE)on a hidden test set, with the primary goal of achieving a lower RMSE than the provided baseline of 11,761 and competing for the top position on the class leaderboard.

### Methodology & Approach

This notebook follows a structured, end-to-end machine learning pipeline to tackle the forecasting problem:

1.  Exploratory Data Analysis (EDA)
2.  Validation Strategy
3.  Feature Engineering
4.  Modeling
5.  Hyperparameter Tuning
6.  Final Forecasting & Submission
---

#### **Set Env**

*   Drive mount
*   Import data
*   Main Vars




In [10]:
# Cell 1: Setup and Data Loading
from google.colab import drive
drive.mount('/content/drive')

# Define path to your data on Google Drive
DATA_PATH = '/content/drive/MyDrive/M5_Project/'

# Load your data
import pandas as pd
train_df = pd.read_csv(DATA_PATH + 'train.csv')
calendar_df = pd.read_csv(DATA_PATH + 'calendar_events.csv')
sample_submission = pd.read_csv(DATA_PATH + 'forecast_submission.csv')


print("Data loaded successfully!")

Mounted at /content/drive
Data loaded successfully!


In [21]:
# First, you might need to install Optuna

import optuna

# General imports
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

from neuralforecast.utils import augment_calendar_df
from hierarchicalforecast.utils import is_strictly_hierarchical
from sklearn.model_selection import train_test_split
import lightgbm as lgb

from math import ceil

from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

warnings.filterwarnings('ignore')

In [ ]:
# ########################### Vars################################################################################
# TARGET = 'sales'         # Our main target
# END_TRAIN = 1941         # Last day in train set
# MAIN_INDEX = ['id','d']  # We can identify item by these columns

##### **Data Merging and Preparation**

We set the data

In [15]:
# Convert 'date' columns to datetime objects
train_df['date'] = pd.to_datetime(train_df['date'])
calendar_df['date'] = pd.to_datetime(calendar_df['date'])

# Merge train_df with calendar_df to get all features in one place
original_df = pd.merge(train_df, calendar_df, on='date', how='left')
original_df['event'].fillna('NoEvent', inplace=True)

print("\nData merged:")
original_df.head()



Data merged:


/tmp/ipython-input-2481678760.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  original_df['event'].fillna('NoEvent', inplace=True)


,store_id,store_name,date,revenue,event
0,0,All Stores,2011-01-29,204126.52,NoEvent
1,0,All Stores,2011-01-30,197426.42,NoEvent
2,0,All Stores,2011-01-31,144267.27,NoEvent
3,0,All Stores,2011-02-01,151903.00,NoEvent
4,0,All Stores,2011-02-02,117399.88,NoEvent


In [18]:
# --- Prepare Data for HierarchicalForecast ---
full_df = original_df.copy()
Y_df = full_df[['date', 'store_id', 'revenue']].copy()
Y_df.rename(columns={'date': 'ds', 'store_id': 'unique_id', 'revenue': 'y'}, inplace=True)
Y_df['unique_id'] = Y_df['unique_id'].astype(str)
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

#Y_df = pd.get_dummies(Y_df, columns=['event'], drop_first=True)


display(Y_df.head())
print(f"Unique IDs being used: {sorted(Y_df['unique_id'].unique(), key=int)}")





,ds,unique_id,y
0,2011-01-29,0,204126.52
1,2011-01-30,0,197426.42
2,2011-01-31,0,144267.27
3,2011-02-01,0,151903.00
4,2011-02-02,0,117399.88


Unique IDs being used: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']


In [19]:
# --- Part B: Create S_df (The Summing Matrix) ---
# The S matrix defines how the bottom-level series sum up to the top levels. The columns of S are the bottom-level series. The rows of S are ALL series (bottom + top).

# Define levels
bottom_level_series = [str(i) for i in range(1, 11)]
top_level_series = ['0']
all_series = top_level_series + bottom_level_series

# Create the matrix as a pandas DataFrame
S_df = pd.DataFrame(0, index=all_series, columns=bottom_level_series)

# The row for the top level ('0') is the sum of all bottom levels. So, set it to 1.
S_df.loc['0', :] = 1

# The rows for the bottom levels are just themselves. This creates an identity matrix in the lower part.
for store_id in bottom_level_series:
    S_df.loc[store_id, store_id] = 1

print("\nS_df (Summing Matrix) constructed manually:")
display(S_df)


S_df (Summing Matrix) constructed manually:


,1,2,3,4,5,6,7,8,9,10
0,1,1,1,1,1,1,1,1,1,1
1,1,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0
5,0,0,0,0,1,0,0,0,0,0
6,0,0,0,0,0,1,0,0,0,0
7,0,0,0,0,0,0,1,0,0,0
8,0,0,0,0,0,0,0,1,0,0
9,0,0,0,0,0,0,0,0,1,0


### Submission data preprocessing

In [22]:
submission_ids = sample_submission['id'].str.split('_', expand=True)
submission_ids.columns = ['store_id_str', 'date_str']
submission_dates = pd.to_datetime(submission_ids['date_str'], format='%Y%m%d')
# The forecast horizon H is the number of unique dates in the submission file
H = submission_dates.nunique()
print(f"The Forecast Horizon (H) is: {H} days.")

The Forecast Horizon (H) is: 92 days.


### Exogenous Features

In [29]:
# Augment calendar features for the historical data.
Y_df_augmented, calendar_cols = augment_calendar_df(df=Y_df, freq='D')

future_dates = pd.date_range(Y_df['ds'].max() + pd.Timedelta(days=1), periods=H, freq='D')
X_future_df = pd.DataFrame({'ds': future_dates})
X_future_df['unique_id'] = 'placeholder'

X_future_df, _ = augment_calendar_df(df=X_future_df, freq='D')

# X_df is the dataframe of exogenous features for the historical period
X_df = Y_df_augmented[calendar_cols].copy()

Y_df = Y_df_augmented[['ds', 'unique_id', 'y']].copy()

print("Created calendar features for past and future dates.")
print("\nHistorical Features (X_df):")
display(X_df.head())
print("\nFuture Features (X_future_df):")
display(X_future_df.head())

Created calendar features for past and future dates.

Historical Features (X_df):


,weekday,monthday,yearday
0,0.333333,0.433333,-0.423288
1,0.500000,0.466667,-0.420548
2,-0.500000,0.500000,-0.417808
3,-0.333333,-0.500000,-0.415068
4,-0.166667,-0.466667,-0.412329



Future Features (X_future_df):


,ds,unique_id,weekday,monthday,yearday
0,2015-10-01,placeholder,0.000000,-0.500000,0.247945
1,2015-10-02,placeholder,0.166667,-0.466667,0.250685
2,2015-10-03,placeholder,0.333333,-0.433333,0.253425
3,2015-10-04,placeholder,0.500000,-0.400000,0.256164
4,2015-10-05,placeholder,-0.500000,-0.366667,0.258904


In [37]:
# tags = {
#     '0': [str(i) for i in range(1, 11)]
# }
# # Here we plot the hierarchical constraints matrix
# hplot = HierarchicalPlot(S=S_df, tags=tags)
# hplot.plot_summing_matrix()

# # plot_series(forecasts_df=Y_df[["unique_id", "ds", "y"]], ids=['TotalAll'])

ValueError: list.remove(x): x not in list

#### Set Train / Validation data

In [32]:

print(f"Using a forecast horizon (H) of: {H} days for the validation split.")

# Determine the cutoff date for the split
split_date = Y_df['ds'].max() - pd.to_timedelta(H, unit='D')

# Split all our dataframes based on this date
Y_train = Y_df[Y_df['ds'] <= split_date]
Y_val = Y_df[Y_df['ds'] > split_date]

X_train = X_df.loc[Y_train.index]
X_val = X_df.loc[Y_val.index]

print(f"Training data from {Y_train['ds'].min().date()} to {Y_train['ds'].max().date()}")
print(f"Validation data from {Y_val['ds'].min().date()} to {Y_val['ds'].max().date()}")
print(f"\nY_train shape: {Y_train.shape}, Y_val shape: {Y_val.shape}")
print(f"X_train shape: {X_train.shape}, X_val shape: {X_val.shape}")

Using a forecast horizon (H) of: 92 days for the validation split.
Training data from 2011-01-29 to 2015-06-30
Validation data from 2015-07-01 to 2015-09-30

Y_train shape: (17754, 3), Y_val shape: (1012, 3)
X_train shape: (17754, 3), X_val shape: (1012, 3)


#### Define Your Hierarchy Structure

In [36]:
# # The top level (store 0) is the sum of all other stores (1 through 10)
# hierarchy_structure = {
#     '0': [str(i) for i in range(1, 11)]
# }

# # In HierarchicalForecast, we define the Summing Matrix 'S' based on this.
# # The library can automatically create this matrix for you from the tags.
# # We just need to define the relationships. Let's create the tags:
# tags = {}
# for node in hierarchy_structure:
#     for bottom_node in hierarchy_structure[node]:
#         if bottom_node not in tags:
#             tags[bottom_node] = []
#         tags[bottom_node].append(node)

# print("Hierarchy tags defined.")
# print(tags)

Hierarchy tags defined.
{'1': ['0'], '2': ['0'], '3': ['0'], '4': ['0'], '5': ['0'], '6': ['0'], '7': ['0'], '8': ['0'], '9': ['0'], '10': ['0']}


## Model

In [40]:
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
# from statsforecast.models import SklearnRegressor
from neuralforecast import NeuralForecast
from hierarchicalforecast.methods import BottomUp, MinTrace

ImportError: cannot import name 'SklearnRegressor' from 'statsforecast.models' (/usr/local/lib/python3.12/dist-packages/statsforecast/models.py)

In [ ]:
best_params = {
    'learning_rate': 0.0443, 'num_leaves': 242, 'max_depth': 10,
    # ... include all other params from your best trial ...
    'objective': 'regression_l1', 'random_state': 42, 'n_jobs':-1,
}

In [ ]:


# --- 1. Let's get your best LGBM parameters from Optuna ---
# (Make sure best_trial.params is available from your tuning step)
best_params = best_trial.params.copy()
# Remove keys that are for Optuna's suggestion process, not for LGBM itself
best_params.pop('n_estimators', None)

# --- 2. Instantiate the HierarchicalForecast object ---
# We will tell it to use your LGBM model for each series.
# The `lags` and `lag_transforms` arguments are how it internally creates features!
# This is a powerful, built-in feature engineering step.
models_to_run = [
    lgb.LGBMRegressor(**best_params)
]

hf = HierarchicalForecast(
    models=models_to_run,
    freq='D', # Daily frequency
    reconciler=MinT(method='wls_struct'), # The MinT reconciler!
    lag_features_kwargs={'lags': [7,14,28], 'agg_fns': ['mean', 'std']}, # Simplified features for this example
    num_threads=4
)

# --- 3. Generate the forecasts ---
# The forecast method will automatically:
# 1. Split data into train/test
# 2. Train a model for each series
# 3. Predict for the horizon H
# 4. Reconcile the predictions
# The library requires Y_df (the dataframe) and S (the summing matrix), which it can build from `tags`.
# Note: For a real prediction, you fit on ALL data and predict on future dates.
# The following is a conceptual example. Let's adapt it for YOUR specific recursive challenge.
# This library is best for when you don't need a complex recursive strategy.
# Since we have one, let's adapt it slightly.

# --- ADAPTED STRATEGY FOR YOUR RECURSIVE MODEL ---
# 1. Generate ALL base forecasts using your existing recursive loop, but now for all 11 series.
#    You need to modify your pipeline to keep store_id=0.
# 2. Once you have the final (incoherent) predictions, format them.
# 3. Use the reconciliation functions on this final output.

# Let's assume you've run your loop and have 'final_incoherent_preds_df'
# with columns ['ds', 'unique_id', 'y_hat'] where y_hat is the model prediction.

# This gets more complex. Let's try the library's simpler, built-in way first.
# It might give a better score due to its more robust feature creation.

cross_validation_results = hf.cross_validation(
    df=hf_df,
    h=H, # Your forecast horizon
    tags=tags,
    step_size=H, # How many steps to jump for each CV fold
    n_windows=2 # Number of cross-validation windows
)

# The result will contain reconciled forecasts.
print("\nCross-validation results with MinT reconciliation:")
display(cross_validation_results.head())

# 4. Modeling with LightGBM

In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# --- 1. Identify Categorical Features ---
# It's important to tell LightGBM which features are categorical.
# This allows it to handle them more efficiently than treating them as numbers.
categorical_features = ['store_id', 'event', 'day_of_week', 'month', 'year', 'is_weekend']

# --- 2. Initialize and Train the Model ---
# We'll start with a solid set of default parameters.
lgbm = lgb.LGBMRegressor(
    objective='rmse',  # Our evaluation metric
    metric='rmse',
    n_estimators=1000, # We'll train up to 1000 trees, but...
    learning_rate=0.05,
    random_state=42,
    n_jobs=-1,         # Use all available CPU cores
    force_col_wise=True
)

print("Training LightGBM model...")
lgbm.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='rmse',
    callbacks=[lgb.early_stopping(10, verbose=True)], # ...early stopping will find the best number for us!
    categorical_feature=categorical_features
)

# --- 3. Make Predictions on the Validation Set ---
val_predictions_log = lgbm.predict(X_val)

# --- 4. Inverse Transform the Predictions ---
# This is a critical step! Our model predicted log(revenue+1).
# We need to convert it back to actual revenue.
val_predictions = np.expm1(val_predictions_log)

# Also inverse transform the true values for comparison
y_val_original = np.expm1(y_val)

# --- 5. Evaluate the Model ---
lgbm_rmse = np.sqrt(mean_squared_error(y_val_original, val_predictions))

print(f"\nSeasonal Naive Baseline RMSE: 4324.50")
print(f"LightGBM Model RMSE: {lgbm_rmse:.2f}")

improvement = (4324.50 - lgbm_rmse) / 4324.50 * 100
print(f"\nImprovement over baseline: {improvement:.2f}%")

if lgbm_rmse < 4324.50:
    print("Excellent! The LightGBM model significantly outperformed our baseline.")
else:
    print("The LightGBM model did not beat the baseline. We may need to review features or hyperparameters.")

In [ ]:
# Plot feature importance
lgb.plot_importance(lgbm, figsize=(12, 8), max_num_features=20)
plt.title('LightGBM Feature Importance')
plt.tight_layout()
plt.show()

### 5. Hyperparameter Tuning

Hyperparameter Tuning with Optuna

In [ ]:


# --- 1. Define the Objective Function for Optuna ---
def objective(trial):
    # --- Define the search space for hyperparameters ---
    # Optuna will pick values from these ranges.
    params = {
        'objective': 'rmse',
        'metric': 'rmse',
        'n_estimators': 1000, # We still use early stopping
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', -1, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'random_state': 42,
        'n_jobs': -1,
    }

    # --- Train the model with the suggested params ---
    model = lgb.LGBMRegressor(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='rmse',
        callbacks=[lgb.early_stopping(10, verbose=False)], # Verbose=False keeps the output clean
        categorical_feature=categorical_features # Re-use our list of categoricals
    )

    # --- Make predictions and calculate RMSE ---
    preds_log = model.predict(X_val)
    preds = np.expm1(preds_log)
    rmse = np.sqrt(mean_squared_error(np.expm1(y_val), preds))

    return rmse

# --- 2. Create and Run the Optuna Study ---
# We want to 'minimize' the RMSE.
study = optuna.create_study(direction='minimize')
# We'll run 50 trials. For a real competition, you might do 100-200.
study.optimize(objective, n_trials=50)


# --- 3. Print the Best Results ---
print("\nOptuna Study Finished!")
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
best_trial = study.best_trial

print("  Value (RMSE): ", best_trial.value)
print("  Params: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

In [ ]:
# --- 4. Train the Final Tuned Model ---
# Get the best hyperparameters from the study
best_params = best_trial.params
best_params['objective'] = 'rmse'
best_params['metric'] = 'rmse'
best_params['random_state'] = 42
best_params['n_estimators'] = 1000 # Use a high number, early stopping will handle it
best_params['n_jobs'] = -1


print("\nTraining final model with best parameters...")
final_model = lgb.LGBMRegressor(**best_params)
final_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='rmse',
    callbacks=[lgb.early_stopping(10, verbose=True)],
    categorical_feature=categorical_features
)

# --- Evaluate the Final Tuned Model ---
final_preds_log = final_model.predict(X_val)
final_preds = np.expm1(final_preds_log)
final_rmse = np.sqrt(mean_squared_error(y_val_original, final_preds))


print(f"Final Tuned LightGBM RMSE: {final_rmse:.2f}")


# The Final Prediction Pipeline

In [ ]:
# --- 1. Retrain the final model on ALL available data ---
# This gives the model maximum information before forecasting.

# First, get the full set of features and the target
ALL_FEATURES = [col for col in df_featured.columns if col not in [TARGET, 'store_name', 'date', 'weekday']]
X_all = df_featured[ALL_FEATURES]
y_all = df_featured[TARGET]

# Use the best parameters found by Optuna
final_model_for_submission = lgb.LGBMRegressor(**best_trial.params)

print("Retraining model on all available data...")
final_model_for_submission.fit(X_all, y_all, categorical_feature=categorical_features)
print("Model retraining complete.")


In [ ]:
df['store_id'].unique()

In [ ]:

# --- 2. Set up the Recursive Forecasting Loop ---
# This is where the magic happens.

# Get the last known date from our original data
last_known_date = df['date'].max()
# Get the last `N` days of data to compute future lags/rolling features.
# A safe number is the max lag or window size we used. Let's use 120 days.
history_df = df_featured.groupby('store_id').tail(120).copy()

future_predictions = []

print(f"\nStarting recursive forecast for {H} days...")
for i in range(H):
    # The date we want to predict
    predict_date = last_known_date + pd.to_timedelta(i + 1, unit='D')

    # Create a placeholder row for each store for the future date
    future_rows = []
    for store_id in df['store_id'].unique():
        future_rows.append({'date': predict_date, 'store_id': store_id})
    future_df = pd.DataFrame(future_rows)

    # Merge calendar events for the future date
    future_df = pd.merge(future_df, calendar_df, on='date', how='left')
    future_df['event'].fillna('NoEvent', inplace=True)

    # Create features for this future row using our past history
    # We combine history with the new rows to calculate lags correctly
    combined_df = pd.concat([history_df, future_df], ignore_index=True)
    featured_future = create_features(combined_df)

    # Isolate the last 10 rows which are the ones we want to predict
    predict_data = featured_future[featured_future['date'] == predict_date].copy()

    # Select the features for prediction
    X_predict = predict_data[ALL_FEATURES]

    # Predict in log scale
    preds_log = final_model_for_submission.predict(X_predict)
    # Inverse transform to original scale
    preds_revenue = np.expm1(preds_log)

    # Store the predictions
    predict_data['revenue'] = preds_revenue
    future_predictions.append(predict_data[['date', 'store_id', 'revenue']])

    # CRITICAL: Update history for the next loop iteration
    # The predicted data becomes part of the new history
    history_df = pd.concat([history_df, predict_data], ignore_index=True)

print("Recursive forecast complete.")

# Combine all predictions into a single dataframe
final_predictions_df = pd.concat(future_predictions, ignore_index=True)



In [ ]:
# Get the last known date from our original data
last_known_date = df['date'].max()
# Get the last `N` days of data to compute future lags/rolling features.
# A safe number is the max lag or window size we used. Let's use 120 days.
history_df = df_featured.groupby('store_id').tail(120).copy()

future_predictions = []

# The date we want to predict
predict_date = last_known_date + pd.to_timedelta(i + 1, unit='D')

# Create a placeholder row for each store for the future date
future_rows = []
for store_id in df['store_id'].unique():
    future_rows.append({'date': predict_date, 'store_id': store_id})
future_df = pd.DataFrame(future_rows)

# Merge calendar events for the future date
future_df = pd.merge(future_df, calendar_df, on='date', how='left')
future_df['event'].fillna('NoEvent', inplace=True)

# Create features for this future row using our past history
# We combine history with the new rows to calculate lags correctly
combined_df = pd.concat([history_df, future_df], ignore_index=True)
featured_future = create_features(combined_df)
featured_future
# Isolate the last 10 rows which are the ones we want to predict
predict_data = featured_future.tail(10)
predict_data
# # Select the features for prediction
# X_predict = predict_data[ALL_FEATURES]

# # Predict in log scale
# preds_log = final_model_for_submission.predict(X_predict)
# # Inverse transform to original scale
# preds_revenue = np.expm1(preds_log)

# # Store the predictions
# predict_data['revenue'] = preds_revenue
# future_predictions.append(predict_data[['date', 'store_id', 'revenue']])

# # CRITICAL: Update history for the next loop iteration
# # The predicted data becomes part of the new history
# history_df = pd.concat([history_df, predict_data], ignore_index=True)


In [ ]:
final_predictions_df


In [ ]:
# --- 3. Format for Submission ---

# We start with our dataframe of predictions for stores 1-10
individual_store_preds = final_predictions_df.copy()
print(f"Generated predictions for {individual_store_preds['store_id'].nunique()} individual stores.")

# Group by date and sum the predictions of stores 1-10 to create the forecast for store 0
aggregate_preds = individual_store_preds.groupby('date')['revenue'].sum().reset_index()
aggregate_preds['store_id'] = 0 # Assign the correct store ID for the aggregate
print("\nCreated aggregate predictions for 'store_id = 0' by summing individual stores.")

# We now have two dataframes: one for stores 1-10, one for store 0. Let's combine them.
# Reorder columns in aggregate_preds to match individual_store_preds for concatenation
aggregate_preds = aggregate_preds[['date', 'store_id', 'revenue']]
# Append the aggregate predictions to the individual ones
all_predictions_df = pd.concat([individual_store_preds.rename(columns={'revenue':'prediction'}), aggregate_preds.rename(columns={'revenue':'prediction'})], ignore_index=True)
all_predictions_df
# Now we create the 'id' column from this complete set of predictions
all_predictions_df['id'] = all_predictions_df['store_id'].astype(str) + '_' + all_predictions_df['date'].dt.strftime('%Y%m%d')

# Merge with the sample submission to ensure correct format and order
submission_df_final = sample_submission[['id']].merge(all_predictions_df[['id', 'prediction']], on='id', how='left')

submission_df_final
# # # --- Final Sanity Check ---
# if submission_df_final['prediction'].isnull().any():
#     print("\nWARNING: There are still NaN values in the submission file! Debugging needed.")
#     # Add a check to see which IDs failed to merge
#     print("IDs that failed to find a match:")
#     print(submission_df_final[submission_df_final['prediction'].isnull()])
# else:
#     print("\nSUCCESS! All 1,012 IDs matched and predictions are filled.")

# Save the submission file
submission_df_final.to_csv( DATA_PATH+ 'submission_df.csv', index=False)

print("\nSubmission file 'submission_df.csv' created successfully!")
print("Here's a sample of the submission file:")
print(submission_df_final.head())

In [ ]:
aggregate_preds

#  Model Explainability with SHAP

In [ ]:
# Install SHAP
!pip install shap -q

import shap

# --- Explain the final model trained on all data ---
# We use a TreeExplainer for tree-based models like LightGBM
explainer = shap.TreeExplainer(final_model_for_submission)
# Calculate SHAP values for the validation set (as a sample)
shap_values = explainer.shap_values(X_val)

# --- Create a SHAP Summary Plot ---
# This is like a super-powered feature importance plot.
# It shows not only the importance but also the direction of the effect.
shap.summary_plot(shap_values, X_val, plot_type="bar", max_display=20) # Classic bar chart
shap.summary_plot(shap_values, X_val, max_display=20) # Beeswarm plot (even better!)

### 3. Final Results records

Date: 29.12
*   **Best Validation RMSE:** '3614.85`
*   **Best Leaderboard RMSE:** `7225.3`
*   **Key Insight:** The model's feature importance plot revealed that high-level calendar features (`month`, `day_of_month`) and `store_id` were the most powerful predictors, indicating strong seasonal and store-level base patterns. Lag and rolling mean features were crucial for capturing recent dynamics.